In [1]:
!pip uninstall tensorflow -y
!pip install kvpress --quiet

In this notebook, we showcase how to use the KVpress pipelines by answering questions about NVIDIA Wikipedia article.

In [2]:
import requests
from bs4 import BeautifulSoup

import torch
from transformers import pipeline

from kvpress import (
    ExpectedAttentionPress,
    KnormPress,
    ObservedAttentionPress,
    RandomPress,
    SnapKVPress,
    StreamingLLMPress
)

# Load the pipeline and data

In [3]:
# Load pipeline

device = "cuda:0"
# ckpt = "meta-llama/Meta-Llama-3.1-8B-Instruct"
ckpt = "Qwen/Qwen2.5-1.5B-Instruct"
# ckpt = "microsoft/Phi-3.5-mini-instruct"
# ckpt = "mistralai/Mistral-Nemo-Instruct-2407"
# attn_implementation = "flash_attention_2"  # use "eager" for ObservedAttentionPress and "sdpa" if you can't use "flash_attention_2"
# use attn_implementation = "eager" for ObservedAttentionPress or attn_implementation = "flash_attention_2" if you can use flash attention
# flash_attention_2 is not fully supported on T4 GPUs, so we are using sdpa
attn_implementation = "sdpa"
pipe = pipeline("kv-press-text-generation", model=ckpt, device=device, torch_dtype=torch.float16, model_kwargs={"attn_implementation":attn_implementation})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [4]:
# Load data
url = "https://en.wikipedia.org/wiki/Nvidia"
content = requests.get(url).content
soup = BeautifulSoup(content, "html.parser")
context = "".join([p.text for p in soup.find_all("p")]) + "\n\n"
tokens = pipe.tokenizer.encode(context, return_tensors="pt").to(device)
print(f"Number of tokens: {tokens.size(1)}")

Number of tokens: 10054


# Use the pipeline with a press

In [5]:
# Pick a press with a compression ratio, you can run the following cells with different presses
compression_ratio = 0.5
press = ExpectedAttentionPress(compression_ratio)
# press = KnormPress(compression_ratio)
# press = RandomPress(compression_ratio)

In [6]:
# Run the pipeline on a single question

question = "Complete this sentence: The Nvidia GeForce Partner Program was a ..."
true_answer = "marketing program designed to provide partnering companies with benefits such as public relations support, video game bundling, and marketing development funds."
pred_answer = pipe(context, question=question, press=press)["answer"]

print(f"Question:   {question}")
print(f"Answer:     {true_answer}")
print(f"Prediction: {pred_answer}")

Question:   Complete this sentence: The Nvidia GeForce Partner Program was a ...
Answer:     marketing program designed to provide partnering companies with benefits such as public relations support, video game bundling, and marketing development funds.
Prediction: The Nvidia GeForce Partner Program was a program that offered discounts and other benefits to partners who sold Nvidia GPUs to customers. It was designed to help Nvidia expand its market share and increase sales of its graphics cards. The program was open to a wide range of


In [7]:
# Run the pipeline on multiple questions, the context will be compressed only once

questions = [
    "What happened on March 1, 2024?",
    "What was the unofficial company motto of Nvidia during the early days?",
]

true_answers = [
    "Nvidia became the third company in the history of the United States to close with a market capitalization in excess of $2 trillion",
    "Our company is thirty days from going out of business",
]

pred_answers = pipe(context, questions=questions, press=press)["answers"]
for question, pred_answer, true_answer in zip(questions, pred_answers, true_answers):
    print(f"Question:   {question}")
    print(f"Answer:     {true_answer}")
    print(f"Prediction: {pred_answer}")
    print()

Question:   What happened on March 1, 2024?
Answer:     Nvidia became the third company in the history of the United States to close with a market capitalization in excess of $2 trillion
Prediction: Based on the information provided, there is no specific event or news item for March 1, 2024 mentioned in the given text. The passage covers Nvidia's history and developments from 1993 to 2024

Question:   What was the unofficial company motto of Nvidia during the early days?
Answer:     Our company is thirty days from going out of business
Prediction: According to the passage, the unofficial company motto of Nvidia during the early days was:

"Our company is thirty days from going out of business."

This quote is attributed to Huang, who began internal presentations to Nvidia staff with those words for many years.



In [8]:
# Use an answer prefix and limit the number of tokens in the answer

question = "What is GTC ?"
true_answer = "Nvidia's GPU Technology Conference (GTC) is a series of technical conferences held around the world."
answer_prefix = "Come on you don't know GTC ? Everyone"
max_new_tokens = 30

pred_answer_with_prefix = pipe(context, question=question, answer_prefix=answer_prefix, press=press, max_new_tokens=max_new_tokens)["answer"]
pred_answer_without_prefix = pipe(context, question=question, press=press, max_new_tokens=max_new_tokens)["answer"]

print(f"Question:              {question}")
print(f"Answer:                {true_answer}")
print(f"Prediction w/o prefix: {pred_answer_without_prefix}")
print(f"Prediction w/ prefix : {answer_prefix + pred_answer_with_prefix}")

Question:              What is GTC ?
Answer:                Nvidia's GPU Technology Conference (GTC) is a series of technical conferences held around the world.
Prediction w/o prefix: GTC stands for GPU Technology Conference. It is an annual conference and exhibition that focuses on the latest developments and advancements in graphics processing technology, particularly in
Prediction w/ prefix : Come on you don't know GTC ? Everyone knows GTC. It's the GPU Technology Conference. It's a major event for the graphics processing unit (GPU) industry. It's where the


In [9]:
# SnapKV use the latest queries to prune the KV-cache. It's hence more efficient if we include the question during compression as the latest queries will correspond to the question.
# However it implies also implies that SnapKV cannot compress well the context independently of the question (e.g. as in a chat use case)


question = "Complete this sentence: In April 2016, Nvidia produced the DGX-1 based on an 8 GPU cluster,"
true_answer = (
    "to improve the ability of users to use deep learning by combining GPUs with integrated deep learning software"
)

press = SnapKVPress(compression_ratio=0.8)

pred_answer_with_question = pipe(context + question, press=press)["answer"]
pred_answer_without_question = pipe(context, question=question, press=press)["answer"]

print(f"Question:         {question}")
print(f"Answer:           {true_answer}")
print(f"Prediction w/ Q:  {pred_answer_with_question}")
print(f"Prediction w/o Q: {pred_answer_without_question}")

Question:         Complete this sentence: In April 2016, Nvidia produced the DGX-1 based on an 8 GPU cluster,
Answer:           to improve the ability of users to use deep learning by combining GPUs with integrated deep learning software
Prediction w/ Q:  In April 2016, Nvidia produced the DGX-1 based on an 8 GPU cluster, to improve the ability of users to use deep learning by combining GPUs with integrated deep learning software.
Prediction w/o Q: In April 2016, Nvidia produced the DGX-1 based on an 8 GPU cluster, which was designed to accelerate machine learning and artificial intelligence tasks. The DGX-1 was a high-performance computing system that used Nvidia's
